In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# reading and loading the data
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Capstone 3 data/bookselling.csv')
df.head()  # displaying the first 5 lines of the dataset

Name  \
0                      10-Day Green Smoothie Cleanse   
1                                  11/22/63: A Novel   
2            12 Rules for Life: An Antidote to Chaos   
3                             1984 (Signet Classics)   
4  5,000 Awesome Facts (About Everything!) (Natio...   

                     Author  User Rating  Reviews  Price  Year        Genre  
0                  JJ Smith          4.7    17350      8  2016  Non Fiction  
1              Stephen King          4.6     2052     22  2011      Fiction  
2        Jordan B. Peterson          4.7    18979     15  2018  Non Fiction  
3             George Orwell          4.7    21424      6  2017      Fiction  
4  National Geographic Kids          4.8     7665     12  2019  Non Fiction


# ***Dataset:*** *Amazon's Top 50 bestselling books from 2009 to 2019.*

---

## **Features:**


*   Name - name of the book
*   Author - author of the book
*   User Rating - User rating from Amazon 
*   Reviews - # of written reviews on amazon 
*   Price - price of the book 
*   Year - years it ranked on the bestseller 
*   Genre - fiction or nonfiction











In [3]:
# shown data types
print(df.dtypes)
# dimension of the data
print(f'Data have {df.shape[0]} records and {df.shape[1]} columns')

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
dtype: object
Data have 550 records and 7 columns



## *Data Description and Summary*

---


In [4]:
df.describe().T

count          mean           std     min     25%     50%  \
User Rating  550.0      4.618364      0.226980     3.3     4.5     4.7   
Reviews      550.0  11953.281818  11731.132017    37.0  4058.0  8580.0   
Price        550.0     13.100000     10.842262     0.0     7.0    11.0   
Year         550.0   2014.000000      3.165156  2009.0  2011.0  2014.0   

                  75%      max  
User Rating      4.80      4.9  
Reviews      17253.25  87841.0  
Price           16.00    105.0  
Year          2017.00   2019.0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


# *Data Cleaning*

---

In [6]:
# changing the data type for Genre
df.Genre = df.Genre.astype('category')
# checking for missing values
df.isnull().sum()

Name           0
Author         0
User Rating    0
Reviews        0
Price          0
Year           0
Genre          0
dtype: int64

There is no missing values, thus, no further tansformations are required.

In [7]:
# However we not using YEAR column for this project
del df['Year']

Now, we are going to remove the duplicates and see how many books are remaining the data.

In [8]:
df = df.drop_duplicates(subset= 'Name', keep = 'last')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 549
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         351 non-null    object  
 1   Author       351 non-null    object  
 2   User Rating  351 non-null    float64 
 3   Reviews      351 non-null    int64   
 4   Price        351 non-null    int64   
 5   Genre        351 non-null    category
dtypes: category(1), float64(1), int64(2), object(2)
memory usage: 16.9+ KB


In [9]:
# Checking for duplicate data and spelling errors (Name column)
y = list(df.select_dtypes(exclude=('int', 'float')).columns)
print(f'Сategorical Columns: {", ".join(y)}.')
for x in y:
    if df[x].duplicated().any() == True:
        print (f'Column {x} has duplicate data.')
    else:
        print (f'Column {x} does have duplicate data.')
for x in y:
    print(f'Actual {x}: "{len(set(df[x]))}" - After Spell Check {x}: "{len(set(df[x].str.title().str.strip()))}"')

df.Name = df.Name.str.title().str.strip()       # fixing the errors

# checking after changes have made.
for x in y:
    print(f'Before {x}: {len(set(df[x]))} After {x}: {len(set(df[x].str.title().str.strip()))}')

Сategorical Columns: Name, Author, Genre.
Column Name does have duplicate data.
Column Author has duplicate data.
Column Genre has duplicate data.
Actual Name: "351" - After Spell Check Name: "350"
Actual Author: "248" - After Spell Check Author: "248"
Actual Genre: "2" - After Spell Check Genre: "2"
Before Name: 350 After Name: 350
Before Author: 248 After Author: 248
Before Genre: 2 After Genre: 2


In [10]:
# Checking duplicates and errors for (Author column)
# fixing the errors 
aut_col = df.Author.sort_values().unique()
aut_col
df = df.replace('George R. R. Martin', 'George R.R. Martin')
df = df.replace('J. K. Rowling', 'J.K. Rowling')
for x in y:
    print(f'Before {x}: {len(set(df[x]))} After {x}: {len(set(df[x].str.title().str.strip()))}')

Before Name: 350 After Name: 350
Before Author: 246 After Author: 246
Before Genre: 2 After Genre: 2


In [11]:
# Checking the Genre column
gen = df.Genre.unique()
gen

['Non Fiction', 'Fiction']
Categories (2, object): ['Fiction', 'Non Fiction']

After cleaning up the dataset, this dataset has 350 different books which are written by 246 authors. And they belong to two different categories, Fiction and Non Fiction. 

# *Data Exploratory*

---

1.   Which book has the most reviews?


2.   Which author's books receive the highest average rating?




In [12]:
# Top 10 books with the # of reviews
top10_book_reviews = df.groupby('Name')[['Reviews']].sum().sort_values('Reviews', ascending=False).head(10).reset_index()
top10_book_reviews
                                                                   

Name  Reviews
0                            Where The Crawdads Sing    87841
1                              The Girl On The Train    79446
2                                           Becoming    61133
3                                          Gone Girl    57271
4                             The Fault In Our Stars    50482
5                           The Nightingale: A Novel    49288
6  Fifty Shades Of Grey: Book One Of The Fifty Sh...    47265
7                                        The Martian    39459
8                        All The Light We Cannot See    36348
9                                      The Alchemist    35799

In [13]:
# Setting the visualization parameters
from plotly.subplots import make_subplots
import plotly.graph_objects as go

chart_1 = make_subplots(rows=1, cols = 1, subplot_titles=("Top 10 books by number of reviews"))


# Setting Bar parameters
chart_1.add_trace(go.Bar(x=top10_book_reviews['Name'], 
                       y=top10_book_reviews['Reviews'],
                       marker_color=['#ff6666','#f47171', '#e87d7d', '#dd8888', '#d19494'],
                       name ='Reviews'),
                       row=1, col=1)

# Setting the parameters of the chart when displaying
chart_1.update_traces(marker_line_color='Green',
                    marker_line_width=1)

chart_1.update_layout(showlegend=False,
                    title_text='Top 10 books by number of reviews', 
                    title_font_size=16, 
                    title_font_family='Arial', 
                    title_x=0.5, 
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

chart_1.show()


'Where The Crawdads Sing' has 87.841k reviews which is the most reviews in Amazon's top 50 bestselling books.

In [14]:
top_authors = df.groupby('Author')[['User Rating']].mean().sort_values('User Rating', ascending=False).head(20).reset_index()
top_authors

# Setting the visualization parameters
top_list = make_subplots(rows=1, cols=1, subplot_titles=("")) # Let's set the number of rows and columns


# Setting table parameters
top_list.add_trace(go.Table(columnorder = [1,2],
                         columnwidth = [100,50],
                         header=dict(values=top_authors.columns, 
                                     line_color='black',
                                     fill_color='lightblue',
                                     height=30),
                         cells=dict(values=[top_authors['Author'],
                                    top_authors['User Rating']], 
                                    line_color='black',
                                    fill_color='yellow')))

# Setting the parameters of the chart when displaying
top_list.update_layout(showlegend=False, 
                    title_text='Top 20 best authors', 
                    title_font_size=16, 
                    title_font_family='Arial', 
                    title_x=0.5,
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

top_list.show()

The highest average user rating is 4.9, and there are 13 authors have the highest average rating accordingly. 

In [15]:
books_by_genre = df.groupby('Genre')[['Name']].count().sort_values('Name', ascending=False).head(10).reset_index()

pie_g = make_subplots(rows=1, cols=1,
                      specs=[[{'type': 'domain'}]],
                      subplot_titles=(""))

# Setting Pie parameters
pie_g.add_trace(go.Pie(values=books_by_genre['Name'],
                       title='Genre',
                       labels=['Non Fiction', 'Fiction'],
                       textinfo='label+percent',
                       insidetextorientation='radial',
                       hole=0.3,
                       marker=dict(colors=['red','orange'], 
                                   line_color='Gray', 
                                   line_width=1),
                       textfont={'color': '#FFFFFF', 'size': 12},
                       textfont_size=12))

# Setting the parameters of the chart when displaying
pie_g.update_layout(margin_t=30, 
                    showlegend=False, 
                    title_text='Count of books genre', 
                    title_x=0.5, 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

pie_g.show()

In [16]:
user_rating = make_subplots(rows=1, cols=1, specs=[[{'type': 'xy'}]])

# Setting Box parameters
user_rating.add_trace(go.Box(x=df['User Rating'], 
                       name='User Rating'))

# Setting the parameters of the Box when displaying
user_rating.update_traces(marker_color='orange')

user_rating.update_layout(showlegend=False, 
                    template='simple_white', 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

user_rating.show()

Plot shows that the median User Rating is 4.6, and its min. is 3.3, while, its max. is 4.9. But less books fall below to the rating of 4.1.

In [17]:
box_reviews = make_subplots(rows=1, cols=1, specs=[[{'type': 'xy'}]])

# Setting Box parameters
box_reviews.add_trace(go.Box(x=df['Reviews'], 
                       name='Reviews'))

# Setting the parameters of the Box when displaying
box_reviews.update_traces(marker_color='orange')

box_reviews.update_layout(showlegend=False, 
                    template='simple_white', 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

box_reviews.show()

the average of reviews for books is 6310, and its max. is 87.841k and its min. is 37 reviews. The range for this box plot is wide, and most of the books receiving reviews is blowing 75 percentile. 

In [18]:
box_price = make_subplots(rows=1, cols=1, specs=[[{'type': 'xy'}]])

# Setting Box parameters
box_price.add_trace(go.Box(x=df['Price'], 
                       name='Price'))

# Setting the parameters of the Box when displaying
box_price.update_traces(marker_color='orange')

# Setting the parameters of the Box when displaying
box_price.update_layout(showlegend=False, 
                    template='simple_white', 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))

# Displaying the Box
box_price.show()

The median for this box plot on the Price is 11, and its max. is 105, meanwhile, its min. is 0, which is free. This is kind of interesting that books with a cost of 0. In other words, books might be given for free, or it could be an error. 

## *1st Hypothesis Test* 

---

**Are genres different by rating?**

Ho: there are no significant differences between genres by rating.

Ha: there are significant differences between genres by rating.

In [19]:
from scipy import stats
from scipy.stats import mannwhitneyu

non_fiction = df[df['Genre'] == 'Non Fiction']['User Rating']
fiction = df[df['Genre'] == 'Fiction']['User Rating']

alpha=0.05

# We pass groups to the criterion for testing
stat, pval = mannwhitneyu(non_fiction, fiction)

print('Statistic:', f'{stat:.3f}')
print('P-Value:', f'{pval:.20f}')
 
# Checking the condition for accepting or rejecting Ho
if pval > alpha:
    print('Accept H0 - There is no significant difference between gneres by rating.')
if pval < alpha:
    print('Reject H0 - There are significant differences between genres by rating.')

Statistic: 13095.000
P-Value: 0.01922648186850501456
Reject H0 - There are significant differences between genres by rating.


In [20]:
import plotly.figure_factory as ff
pic_1 = ff.create_distplot([non_fiction, fiction], 
                         ['Non_fiction', 'Fiction'], 
                         colors=['blue','orange',],
                         bin_size=.1, 
                         show_rug=False)

# Add title
pic_1.update_layout(title_text='Сomparison of the distribution density in the group Fiction and the group Non_fiction', 
                     template='simple_white',
                     font=dict(family='Arial', size=12, color='black'))
pic_1.show()

**TEST 1 CONCLUSION:** 

 According to the result of Mann-Whitney test, Ho is rejected, thus, there is statistically significant differences were obtained between the genres by rating. In other words, users evaluate books differently depending on the genre. 

## *2nd Hypothesis Test* 

---

**Does the Genre of the book affect its Price?**

Ho: there is no significant relationship between the Prices and the Genres.

Ha: there exists a significant relationship between the Prices and the Genres.


In [24]:
# Generate samples to test

from scipy import stats
from scipy.stats import mannwhitneyu

df["Genre"] = df["Genre"].replace({"Fiction": 1, "Non Fiction": 0})
x1 = df[df['Genre'] == 0]['Price']
y1 = df[df['Genre'] == 1]['Price']

# Set the alpha level
alpha2= 0.5

# We pass groups to the criterion for testing using the nonparametric Mann-Whitney test.
stat2, pval2 = mannwhitneyu(x1, y1)

print('Statistic:', f'{stat2:.3f}')
print('P-Value:', f'{pval2:.20f}')
 
# Checking the condition for accepting or rejecting H0
if pval2 > alpha2:
    print('Accepting Ho, there is no significant relationship.')
if pval2 < alpha2:
    print('Rejecting Ho, there exists a significant relationship')

Statistic: 17490.500
P-Value: 0.01941239390458456351
Rejecting Ho, there exists a significant relationship


In [25]:
import plotly.figure_factory as ff
pic_2 = ff.create_distplot([x1, y1], 
                         ['Non_fiction', 'Fiction'], 
                         colors=['blue','orange',],
                         bin_size=.1, 
                         show_rug=False)

# Add title
pic_2.update_layout(title_text='Сomparison of the group Fiction and the group Non_fiction', 
                     template='simple_white',
                     font=dict(family='Arial', size=12, color='black'))
pic_2.show()

**TEST 2 CONCLUSION:**

From the non-parametric Mann-Whitney test, Ho is rejected. As a result, we positively say that, there exists a significant relationship between the Prices and the Genres. In other words, the Genre of the book has significant impact on the Price at which book is sold. 

## *Recommendations*

---

Overall, we conclude that Amazon's books selling is highly depending on users' ratings and its prices of the books which are sold. Additionally, Non-fiction is more likely to become a bestseller. As a result, Amazon should have some changes on its prices, especially on the Non-fiction genre. 